In [8]:
import os
import sys
from pathlib import Path

root = Path(os.getcwd()).parent
sys.path.append(str(root))

import pandas as pd

# Load morphological data
morph_df = pd.read_csv("../data/morphology_data_cleaned.csv")
morph_df.rename(columns={"id": "neighborhood_id"}, inplace=True)

# Load health data
health_df_soc = pd.read_excel(
    "../data/synthetic_health_data.xlsx", sheet_name="Participant_SocioDemograph_Data"
)
health_df_clin = pd.read_excel(
    "../data/synthetic_health_data.xlsx", sheet_name="Participant_HEALTH_Data"
)

# Merge health data
health_df = pd.merge(
    health_df_soc, health_df_clin, on=["participant_id", "neighborhood_id"], how="inner"
)

# Merge morphology and health data
df = pd.merge(morph_df, health_df, on="neighborhood_id", how="inner")

print(f"Final dataset shape: {df.shape}")

Final dataset shape: (2631, 33)


In [9]:
# Create quantile-based age groups so each bin has comparable counts
age_bin_series = pd.qcut(
    df["age"],
    q=min(4, df["age"].nunique()),
    duplicates="drop",
)

age_labels = []
age_intervals = age_bin_series.cat.categories
for idx, interval in enumerate(age_intervals):
    left_edge = interval.left
    right_edge = interval.right
    if idx == 0:
        label = f"<= {int(round(right_edge))}"
    elif idx == len(age_intervals) - 1:
        label = f"> {int(round(left_edge))}"
    else:
        label = f"{int(round(left_edge))}-{int(round(right_edge))}"
    age_labels.append(label)

df["age_bin"] = age_bin_series.cat.rename_categories(age_labels)

In [10]:
numerical_cols = [
    "PC1",
    "PC2",
    "PC3",
    "PC4",
    "PC5",
    "PC6",
    "z_distance",
    "lst_mean",
    "solar_summ",
    "solar_wint",
    "pm10_mean",
    "pm25_mean",
    "no2_mean",
    "noiseday_m",
    "noisenight",
    "points_sleep_deprivation",
    "sleeping_hours",
    "bedtime_hour",
]

categorical_cols = [
    "typology",
    "sex",
    "income",
    "education_level",
    "age_bin",
    "GHQ12_score",
]

binary_cols = [
    "heart_failure",
    "heart_rhythm",
    "sleep_disorder_hot",
    "d_breath_respiratory",
    "d_breath_asthma",
]

possible_targets = [
    # Sleep and Circadian health
    "points_sleep_deprivation",
    "sleeping_hours",
    "sleep_disorder_hot",
    "bedtime_hour",
    # Mental health
    "GHQ12_score",
    # Cardiovascular diseases
    "heart_failure",
    "heart_rhythm",
    # Respiratory conditions
    "d_breath_respiratory",
    "d_breath_asthma",
]

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Drop columns that are neither numerical nor categorical nor binary
features = df.drop(
    columns=[
        col
        for col in df.columns
        if col not in numerical_cols + categorical_cols + binary_cols
        or col in possible_targets
    ]
)

# Encode categorical variables
for col in categorical_cols:
    if col not in possible_targets:
        dummies = pd.get_dummies(features[col], prefix=col)
        features = pd.concat([features.drop(columns=[col]), dummies], axis=1)

In [12]:
features.columns

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'lst_mean', 'solar_summ',
       'solar_wint', 'pm10_mean', 'pm25_mean', 'no2_mean', 'noiseday_m',
       'noisenight', 'typology_A', 'typology_B', 'typology_C', 'typology_D',
       'typology_E', 'typology_F', 'typology_G', 'typology_I', 'typology_J',
       'sex_Female', 'sex_Male', 'income_High', 'income_Low', 'income_Medium',
       'education_level_Bachelor', 'education_level_High School',
       'education_level_Master', 'education_level_PhD', 'age_bin_<= 33',
       'age_bin_33-49', 'age_bin_49-63', 'age_bin_> 63'],
      dtype='object')

In [13]:
features.head()

,PC1,PC2,PC3,PC4,PC5,PC6,lst_mean,solar_summ,solar_wint,pm10_mean,...,income_Low,income_Medium,education_level_Bachelor,education_level_High School,education_level_Master,education_level_PhD,age_bin_<= 33,age_bin_33-49,age_bin_49-63,age_bin_> 63
0,0.526055,0.33848,0.846051,-1.420268,-0.217506,0.687111,-0.111053,1.130211,0.982144,-1.361464,...,True,False,True,False,False,False,False,True,False,False
1,0.526055,0.33848,0.846051,-1.420268,-0.217506,0.687111,-0.111053,1.130211,0.982144,-1.361464,...,False,False,False,False,False,True,False,False,False,True
2,0.526055,0.33848,0.846051,-1.420268,-0.217506,0.687111,-0.111053,1.130211,0.982144,-1.361464,...,False,True,False,False,False,True,False,False,True,False
3,0.526055,0.33848,0.846051,-1.420268,-0.217506,0.687111,-0.111053,1.130211,0.982144,-1.361464,...,False,False,False,False,False,True,False,False,True,False
4,0.526055,0.33848,0.846051,-1.420268,-0.217506,0.687111,-0.111053,1.130211,0.982144,-1.361464,...,True,False,False,True,False,False,False,False,False,True


In [14]:
# Define target variable
target_feature = "GHQ12_score"
target = df[target_feature]

if target_feature in numerical_cols:
    # Normalize target
    target = (target - target.mean()) / target.std()
    model_type = "regression"
elif target_feature in categorical_cols:
    model_type = "multi-class classification"
elif target_feature in binary_cols:
    model_type = "binary classification"

In [15]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

# Initialize and train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [16]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 17.233576728500807
R^2 Score: -0.1682357077760277
